## Analyze trial results

In [74]:
import pandas as pd
import os, sys
import os.path
import numpy as np
import scipy
import scipy.stats

In [51]:
trial_end_date = 1000
run_output_dir = '2018-04-30--03.43.47-2036727724__DA3__n=600'
file_signature = 'populations.csv'

In [15]:
pops_file = [s for s in os.listdir(run_output_dir) if s.find(file_signature)>-1][0]

In [85]:
header_output = pd.read_csv(os.path.join(run_output_dir, pops_file), nrows=1, header=1)

### Study details

In [89]:
header_output[['vaccine_schedule', 'initial_pwid_count', 'vaccine_study_arm_n']]

,vaccine_schedule,initial_pwid_count,vaccine_study_arm_n
0,D3a,32000,600


In [91]:
trial_end_date

1000

### Populations

In [90]:
pops_output = pd.read_csv(os.path.join(run_output_dir, pops_file), header=3)

In [45]:
pops_output.columns

Index(['BurnInMode', 'Tick', 'Year', 'activations_daily', 'aggregate_courses',
       'aggregate_posttreat', 'cured_daily', 'incidence_daily', 'losses_daily',
       'treatment_recruited_daily',
       ...
       'fraction_VaccineArm=noarm', 'hcvabpos_VaccineArm=noarm',
       'infected_VaccineArm=noarm', 'chronic_VaccineArm=noarm',
       'population_VaccineArm=noarm', 'ABpreval_VaccineArm=noarm',
       'RNApreval_VaccineArm=noarm', 'intreatment_VaccineArm=noarm',
       'vaccinetrial_VaccineArm=noarm', 'Unnamed: 298'],
      dtype='object', length=299)

In [68]:
trial_pops=np.array(pops_output.loc[trial_end_date, ['vaccinetrial_VaccineArm=study', 'vaccinetrial_VaccineArm=placebo']])
pops_output.loc[trial_end_date, ['Tick', 'vaccinetrial_VaccineArm=study', 'vaccinetrial_VaccineArm=placebo']]

Tick                               1001.001
vaccinetrial_VaccineArm=study        75.000
vaccinetrial_VaccineArm=placebo      89.000
Name: 1000, dtype: float64

### Infection status

In [69]:
chronic_pops=np.array(pops_output.loc[trial_end_date, ['chronic_VaccineArm=study', 'chronic_VaccineArm=placebo']])
pops_output.loc[trial_end_date, ['Tick', 'chronic_VaccineArm=study', 'chronic_VaccineArm=placebo']]

Tick                          1001.001
chronic_VaccineArm=study        16.000
chronic_VaccineArm=placebo      49.000
Name: 1000, dtype: float64

### results

In [92]:
scipy.stats.chi2_contingency([1,2,3,4])

(0.0, 1.0, 0, array([ 1.,  2.,  3.,  4.]))

In [97]:
notchronic_pops = trial_pops - chronic_pops

In [120]:
trial_contingency_table = np.reshape(np.concatenate((trial_pops, chronic_pops), axis=0), newshape=(2,2))

In [121]:
chi2, p, dof, expected = scipy.stats.chi2_contingency(trial_contingency_table)
chi2, p, expected

(7.8084276966134176,
 0.0052003130855061228,
 array([[ 65.17030568,  98.82969432],
        [ 25.82969432,  39.17030568]]))

## Vaccine efficacy:
$$ VE_𝑎 = 1 − 𝑎_V/𝑎_𝑝 $$

In our case, we are interested in the prevention of chronic infection.  Therefore, we calculate:
$$VE_a = 1 - $$(HCV chronic vaccinees / total vaccinees)/(HCV chronic placebo / total placebo)

In [137]:
a_p = (pops_output.loc[trial_end_date, 'chronic_VaccineArm=placebo'] / pops_output.loc[trial_end_date, 'population_VaccineArm=placebo'])
a_p

0.085814360770577927

In [138]:
a_v = (pops_output.loc[trial_end_date, 'chronic_VaccineArm=study'] / pops_output.loc[trial_end_date, 'population_VaccineArm=study'])
a_v

0.028419182948490232

In [139]:
1 - (a_v/a_p)

0.66882952115126693

Extended trial - 2x normal duration

In [140]:
a_p2 = (pops_output.loc[trial_end_date*2, 'chronic_VaccineArm=placebo'] / pops_output.loc[trial_end_date*2, 'population_VaccineArm=placebo'])
a_p2

0.17375231053604437

In [141]:
a_v2 = (pops_output.loc[trial_end_date*2, 'chronic_VaccineArm=study'] / pops_output.loc[trial_end_date*2, 'population_VaccineArm=study'])
a_v2

0.067164179104477612

In [143]:
1 - (a_v2/a_p2)

0.61344871387742139

# Conclusions

* The trial achieved statistically significant reduction
* The trial failed to achieve 80% reduction in chronicity, attaining only 60-67%